In [1503]:
import pyodbc
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import style
style.use('fivethirtyeight')

In [1504]:
# Connect to the Database
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

# QUERY 01: Operations

In [1505]:
df = pd.read_sql_query("SELECT CONVERT(varchar,t.completedDateTime,100) Date, w.name warehouse, p.name project, oc.name operationCode, t.employee\
    FROM datex_footprint.Tasks t\
    JOIN datex_footprint.OperationCodes oc\
        ON oc.id = t.operationCodeId\
    JOIN datex_footprint.Warehouses w\
        ON w.id = t.warehouseId\
    JOIN datex_footprint.Projects p\
        ON p.id = t.projectId\
    WHERE (cast(t.completedDateTime as Date) = cast(getdate() as Date)) AND (t.employee != 'FootPrintService') AND (t.employee != 'Foot Print Service')\
    ORDER BY completedDateTime DESC", cnxn)

In [1506]:
# Adding Hours
df['Hour'] = pd.to_datetime(df['Date']).dt.hour
df.head()

,Date,warehouse,project,operationCode,employee,Hour
0,Mar 29 2022 5:54PM,10,ISDIN Corporation,Receiving,Sandra Vega,17
1,Mar 29 2022 5:54PM,10,ISDIN Corporation,Receiving,Sandra Vega,17
2,Mar 29 2022 5:54PM,10,ISDIN Corporation,Receiving,Sandra Vega,17
3,Mar 29 2022 5:54PM,10,ISDIN Corporation,Receiving,Sandra Vega,17
4,Mar 29 2022 5:54PM,10,ISDIN Corporation,Picking,Hugo Gonzales,17


### Warehouse Activity

In [1507]:
df_warehouse = df.groupby(['warehouse']).count()[['operationCode']]
df_warehouse.reset_index(inplace=True)
df_warehouse.head()

,warehouse,operationCode
0,10,1008
1,102,58
2,12,69
3,15,339
4,16,293


In [1508]:
# Pie Chart
warehouse = df_warehouse['warehouse']
activity = df_warehouse['operationCode']

#plt.subplots(figsize=(8,8))
#plt.pie(activity, labels=warehouse)
#plt.show()

In [1509]:
# Bar Chart
warehouse = df_warehouse['warehouse']
activity = df_warehouse['operationCode']

#plt.subplots(figsize=(8,8))
#plt.bar(warehouse, activity)
#plt.show()

In [1510]:
# Save Data in JSON format
df_warehouse_index = df_warehouse.set_index(['warehouse'])
df_warehouse_index.to_json('data/warehouse.json')
df_warehouse_index.head()

,operationCode
warehouse,
10,1008
102,58
12,69
15,339
16,293


### All Operations type

In [1511]:
df_operation = df.groupby(['operationCode']).count()[['project']]
df_operation.reset_index(inplace=True)
df_operation.head()

,operationCode,project
0,Batch Move,165
1,Count,77
2,Create Inventory,5
3,DONE RECEIVING,23
4,DRIVER CHECK IN,46


In [1512]:
# Horizontal Bar Chart
values = df_operation['project']
operation = df_operation['operationCode']

#plt.subplots(figsize=(8,8))
#plt.barh(operation, values)
#plt.gca().invert_yaxis()
#plt.show()

In [1513]:
df_operation.rename(columns={"project": "Tasks"})
df_operation.head()

,operationCode,project
0,Batch Move,165
1,Count,77
2,Create Inventory,5
3,DONE RECEIVING,23
4,DRIVER CHECK IN,46


In [1514]:
# Save Data in JSON format
df_operation = df_operation.rename(columns={"project": "Tasks"})
df_operation_index = df_operation.set_index(['operationCode'])
df_operation_index.to_json('data/operations_type.json')
df_operation_index.head()

,Tasks
operationCode,
Batch Move,165
Count,77
Create Inventory,5
DONE RECEIVING,23
DRIVER CHECK IN,46


### Excluding Operations PICKING and RECEIVING

In [1515]:
df_operation_excluded = df_operation.loc[(df_operation["operationCode"] != 'Picking') &
                                         (df_operation["operationCode"] != 'Receiving')]

In [1516]:
# Save Data in JSON format
df_operation_excluded = df_operation_excluded.rename(columns={"project": "Tasks"})
df_operation_excluded_index = df_operation_excluded.set_index(['operationCode'])
df_operation_excluded_index.to_json('data/operations_type_excluded.json')
df_operation_excluded_index.head()

,Tasks
operationCode,
Batch Move,165
Count,77
Create Inventory,5
DONE RECEIVING,23
DRIVER CHECK IN,46


### Operations per Hour

In [1517]:
df_hours = df.groupby(['Hour']).count()[['operationCode']]
df_hours.reset_index(inplace=True)
df_hours.head()

,Hour,operationCode
0,6,8
1,7,45
2,8,288
3,9,379
4,10,371


In [1518]:
# Bar Chart
hour = df_hours['Hour']
activity = df_hours['operationCode']

#plt.subplots(figsize=(8,8))
#plt.bar(hour, activity)
#plt.xticks(hour)
#plt.show()

In [1519]:
df_hours = df_hours.rename(columns={'operationCode': 'Operations'})
df_hours_index = df_hours.set_index(['Hour'])
df_hours_index.to_json('data/hours.json')
df_hours_index.head()

,Operations
Hour,
6,8
7,45
8,288
9,379
10,371


# PICKING

In [1520]:
# Filtered by Picking
df_picking = df[df['operationCode'] == 'Picking']

In [1521]:
# Grouped by Employee and Hour
df_picking_grouped = df_picking.groupby(['employee', 'Hour']).count()[['operationCode']]
df_picking_grouped.reset_index(inplace=True)
df_picking_grouped.head()

,employee,Hour,operationCode
0,Angel Luna,8,4
1,Angel Luna,9,9
2,Angel Luna,10,5
3,Angel Luna,11,11
4,Angel Luna,12,4


In [1522]:
# Pivot
df_picking_pivoted = df_picking_grouped.pivot(index='employee', columns='Hour', values='operationCode')
df_picking_pivoted = df_picking_pivoted.fillna(0)
df_picking_pivoted = df_picking_pivoted.rename_axis(None, axis=1)
#df_picking_pivoted.reset_index(inplace=True)

df_picking_pivoted.head()

,7,8,9,10,11,12,13,14,15,16,17
employee,,,,,,,,,,,
Angel Luna,0.0,4.0,9.0,5.0,11.0,4.0,13.0,0.0,1.0,0.0,0.0
Angela Pena,0.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Brad Bacchetti,0.0,6.0,17.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Carl Ambroise,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Carlos Gomez,0.0,12.0,0.0,5.0,11.0,0.0,0.0,7.0,5.0,6.0,9.0


### Save in JSON File

In [1523]:
# Save into JSON
pickers = df_picking_pivoted.T
pickers.to_json('data/pickers.json')
pickers.head()

employee,Angel Luna,Angela Pena,Brad Bacchetti,Carl Ambroise,Carlos Gomez,Carly Potts,Chris Martin,Djenyson Sanon,Edwin Perez,Ernest Ferguson,...,Reggie Bracken,Sergio Piura,Shannon Williamson,Shawn Saunders,Sujelin Soto-Orta,Tyler Gonzales,Wendy Medrano,William Trevino Trevino,Yaima Castillo,yudisel tubella
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0
8,4.0,0.0,6.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,20.0,0.0,29.0,0.0,0.0,0.0
9,9.0,45.0,17.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,...,0.0,1.0,22.0,25.0,10.0,0.0,32.0,0.0,0.0,0.0
10,5.0,0.0,14.0,2.0,5.0,0.0,10.0,0.0,0.0,2.0,...,0.0,3.0,0.0,0.0,34.0,0.0,30.0,0.0,29.0,12.0
11,11.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,10.0,0.0,13.0,0.0,0.0,0.0


# Picking By Warehouse Function

In [1524]:
# Picking Function
def picking_by_warehouse (warehouse):
    # Filter by warehouse
    df_w = df_picking[df_picking['warehouse'] == warehouse]
    
    # Grouped by Employee and Hour
    df_w_group = df_w.groupby(['employee', 'Hour']).count()[['operationCode']]
    df_w_group.reset_index(inplace=True)
    
    # Pivot
    df_w_pivot = df_w_group.pivot(index='employee', columns='Hour', values='operationCode')
    df_w_pivot = df_w_pivot.fillna(0)
    df_w_pivot = df_w_pivot.rename_axis(None, axis=1)
    
    # Save into JSON
    pickers_df = df_w_pivot.T
    file_name = f'data/pickers_w{warehouse}.json'
    pickers_df.to_json(file_name)

In [1525]:
# Save warehouse data into JSON files
picking_by_warehouse('10')
picking_by_warehouse('12')
picking_by_warehouse('15')
picking_by_warehouse('16')
picking_by_warehouse('18')
picking_by_warehouse('19')
picking_by_warehouse('20')
picking_by_warehouse('21')
picking_by_warehouse('22')

# Receiving

In [1526]:
# Filtered by Picking
receiving_df = df[df['operationCode'] == 'Receiving']
receiving_df.head()

,Date,warehouse,project,operationCode,employee,Hour
0,Mar 29 2022 5:54PM,10,ISDIN Corporation,Receiving,Sandra Vega,17
1,Mar 29 2022 5:54PM,10,ISDIN Corporation,Receiving,Sandra Vega,17
2,Mar 29 2022 5:54PM,10,ISDIN Corporation,Receiving,Sandra Vega,17
3,Mar 29 2022 5:54PM,10,ISDIN Corporation,Receiving,Sandra Vega,17
22,Mar 29 2022 5:48PM,22,FUJIFILM Diosynth Biotechnologies Texas LLC,Receiving,William Trevino Trevino,17


In [1527]:
# Grouped by Employee and Hour
receiving_group_df = receiving_df.groupby(['employee', 'Hour']).count()[['operationCode']]
receiving_group_df.reset_index(inplace=True)
receiving_group_df.head()

,employee,Hour,operationCode
0,Andre Nau,16,22
1,Angela Briceno,16,2
2,Angela Pena,12,3
3,Angela Pena,15,5
4,Angela Pena,16,24


In [ ]:
# Pivot
receiving_pivot_df = receiving_group_df.pivot(index='employee', columns='Hour', values='operationCode')
receiving_pivot_df = receiving_pivot_df.fillna(0)
receiving_pivot_df = receiving_pivot_df.rename_axis(None, axis=1)
#df_picking_pivoted.reset_index(inplace=True)

receiving_pivot_df.head()

,6,7,8,9,10,11,12,13,14,15,16,17
employee,,,,,,,,,,,,
Andre Nau,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,0.0
Angela Briceno,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
Angela Pena,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,24.0,0.0
Brad Bacchetti,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,41.0,0.0,0.0
Carl Ambroise,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Save into JSON
receivers = receiving_pivot_df.T
receivers.to_json('data/receivers.json')
receivers.head()

# QUERY 02: Open Outbounds

In [ ]:
df_02 = pd.read_sql_query("SELECT DISTINCT w.name warehouse, p.name project, s.lookupCode shipment_number, s.createdSysDateTime\
    FROM datex_footprint.shipments s\
    LEFT JOIN datex_footprint.ShipmentOrderLookup sol\
        on sol.shipmentId = s.id\
    LEFT JOIN datex_footprint.orders o\
        on o.id = sol.orderId\
    LEFT JOIN datex_footprint.Projects p\
        on p.id = o.projectId\
    LEFT JOIN datex_footprint.warehouses w\
        on w.id = isnull(s.actualWarehouseId, s.expectedWarehouseId)\
    WHERE s.statusId in (1, 2, 4) and s.typeId = 2\
    ORDER BY createdSysDateTime", cnxn)

In [ ]:
df_02.head()

In [ ]:
df_02['age_order'] = pd.Timestamp.now() - df_02['createdSysDateTime']
df_02.head()

In [ ]:
df_02['age_order_days'] = df_02['age_order'].dt.components['days']
df_02['age_order_hours'] = df_02['age_order'].dt.components['hours']
df_02.head()

In [ ]:
bins =[-1,1,2,3,7,30,60, 90,365, 730,10000]
bin_names = ['24 Hours', '48 Hours', '72 Hours', '1 Week', '1 Month', '2 Months', '3 Months', '1 Year', '2 Years', 'More than 2 years']

df_02['age_group'] = pd.cut(df_02['age_order_days'], bins, labels=bin_names)

In [ ]:
df_02_grouped = df_02.groupby(['age_group'])[['project']].count()

In [ ]:
df_02_grouped = df_02_grouped.rename(columns={'project': 'orders'})

In [ ]:
df_02_grouped

### Save in JSON File

In [ ]:
open_outbounds = df_02_grouped
open_outbounds.to_json('data/open_outbounds.json')
open_outbounds

In [ ]:
df_02_grouped = df_02_grouped.iloc[::-1]
df_02_grouped.plot.barh()

In [ ]:
# Close connection
cnxn.close()